In [1]:
# coding: utf-8

# In[41]:


from __future__ import print_function
import numpy as np
import keras
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import gym
import matplotlib.pyplot as plt

# initialize variables
n_actions = 2


env = gym.make("CartPole-v0")
observation = env.reset()

Using TensorFlow backend.
[2018-01-01 22:43:32,996] Making new env: CartPole-v0


In [2]:
input1 = keras.layers.Input(shape=(4,))
input2 = keras.layers.Input(shape=(2,))
concat = keras.layers.Concatenate(axis=-1)([input1,input2])
dense1 = Dense(64, activation='relu')(concat)
out = keras.layers.Dense(1, activation='linear')(dense1)
model = keras.models.Model(inputs=[input1, input2], outputs=out)
model.summary()

model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.SGD(lr=0.001))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 6)            0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           448         concatenate_1[0][0]              
__________

In [11]:
def Q(s,n_a):
    """takes the state, and the number of actions. Returns a numpy array of estimates of Q(s,a)"""
    q = np.zeros(n_a)
    for i in range(n_a):
        a = np.zeros(n_a)
        a[i] = 1
        q[i] = model.predict([s,a.reshape(1,2)])
    return q


def act(Q):
    """pick action based on Q"""
    epsilon = 0.0001
    if np.sum(np.absolute(Q))< epsilon:
        return np.random.randint(2)
    elif np.random.uniform()<0.7:
        if Q[1]>Q[0]:
            return 1
        else:
            return 0
    else:
        return np.random.randint(2)



def process_reward(r):
    """ take 1D float array of rewards and compute discounted reward """
    r = np.ones_like(r)*r.size
    return r


In [12]:
states,actions,rewards = [],[],[]
episode_number = 0


render = False
epi_reward = None
while True:
    # 1 iter per frame
    s = observation.reshape(1,4)
    # choose action, apply, get measurements
    q = Q(s,n_actions)
    a = act(q)
    observation, reward, done, info = env.step(a)
    states.append(s)
    action = np.zeros(n_actions)
    action[a] = 1
    actions.append(action)
    rewards.append(reward)
    if done:
        episode_number += 1
        print(np.sum(rewards))
        eps = np.vstack(states)
        epa = np.vstack(actions)
        epr = process_reward(np.vstack(rewards))
        # print(epr)
        states,actions,rewards = [],[],[]
        model.fit(x=[eps,epa],y=epr,epochs=1,verbose=0)
        if episode_number%100==0:
            model.save("model.h5")
        observation = env.reset()

[2018-01-01 22:50:31,879] You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


0.0
20.0
16.0
16.0
17.0
24.0
15.0
12.0
17.0
9.0
11.0
17.0
22.0
9.0
8.0
11.0
11.0
10.0
13.0
10.0
11.0
9.0
12.0
9.0
16.0
14.0
13.0
10.0
14.0
12.0
9.0
9.0
10.0
10.0
11.0
10.0
12.0
12.0
14.0
11.0
19.0
9.0
10.0
10.0
14.0
12.0
11.0
10.0
11.0
8.0
11.0
10.0
15.0
10.0
10.0
14.0
13.0
10.0
19.0
9.0
9.0
9.0
16.0
14.0
13.0
10.0
12.0
11.0
12.0
14.0
10.0
10.0
11.0
9.0
8.0
12.0
9.0
12.0
11.0
9.0
11.0
14.0
15.0
11.0
11.0
15.0
11.0
12.0
9.0
9.0
11.0
10.0
11.0
13.0
9.0
14.0
18.0
10.0
10.0
14.0
11.0
10.0
10.0
9.0
9.0
9.0
9.0
12.0
10.0
10.0
18.0
10.0
9.0
13.0
9.0
12.0
9.0
8.0
12.0
10.0
11.0
12.0
12.0
10.0
9.0
9.0
10.0
14.0
12.0
12.0
9.0
9.0
11.0
13.0
13.0
25.0
10.0
9.0
11.0
10.0
10.0
13.0
11.0
10.0
12.0
9.0
11.0
13.0
12.0
14.0
16.0
8.0
9.0
12.0
12.0
11.0
10.0
11.0
11.0
10.0
14.0
11.0
10.0
13.0
14.0
11.0
14.0
14.0
10.0
8.0
14.0
9.0
20.0
12.0
10.0
10.0
11.0
12.0
11.0
10.0
11.0
10.0
15.0
12.0
14.0
10.0
9.0
9.0
9.0
9.0
10.0
9.0
17.0
10.0
12.0
10.0
11.0
9.0
15.0
10.0
10.0
14.0
8.0
15.0
12.0
9.0
18.0
11.0
10.0
9

KeyboardInterrupt: 

In [9]:
q

array([-0.38179839, -0.38992131])